<a href="https://colab.research.google.com/github/jonasfelipe22/CD_indicum/blob/main/CD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prova de Seleção Jonas Felipe Barros Silva 31994912913

In [43]:
pip install MultiColumnLabelEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!pip install pyforest 
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [218]:
import pandas as pd
df=pd.read_csv ('desafio_manutencao_preditiva_treino.csv')
dfo=pd.read_csv ('desafio_manutencao_preditiva_treino.csv')
print(df)

        udi product_id type  air_temperature_k  process_temperature_k  \
0         1     M14860    M             298.10                 308.60   
1         2     L47181    L             298.20                 308.70   
2         5     L47184    L             298.20                 308.70   
3         6     M14865    M             298.10                 308.60   
4         7     L47186    L             298.10                 308.60   
...     ...        ...  ...                ...                    ...   
6662   9995     L57174    L             298.80                 308.30   
6663   9996     M24855    M             298.80                 308.40   
6664   9997     H39410    H             298.90                 308.40   
6665   9999     H39412    H             299.00                 308.70   
6666  10000     M24859    M             299.00                 308.70   

      rotational_speed_rpm  torque_nm  tool_wear_min failure_type  
0                     1551      42.80              0   

In [219]:
df.isnull().sum()


udi                      0
product_id               0
type                     0
air_temperature_k        0
process_temperature_k    0
rotational_speed_rpm     0
torque_nm                0
tool_wear_min            0
failure_type             0
dtype: int64

In [220]:
corr_df = df.corr()
print(corr_df)

                        udi  air_temperature_k  process_temperature_k  \
udi                    1.00               0.12                   0.33   
air_temperature_k      0.12               1.00                   0.88   
process_temperature_k  0.33               0.88                   1.00   
rotational_speed_rpm   0.00               0.04                   0.03   
torque_nm             -0.00              -0.02                  -0.02   
tool_wear_min         -0.00               0.02                   0.02   

                       rotational_speed_rpm  torque_nm  tool_wear_min  
udi                                    0.00      -0.00          -0.00  
air_temperature_k                      0.04      -0.02           0.02  
process_temperature_k                  0.03      -0.02           0.02  
rotational_speed_rpm                   1.00      -0.88          -0.00  
torque_nm                             -0.88       1.00          -0.00  
tool_wear_min                         -0.00      -0.00  

In [221]:
df.columns

Index(['udi', 'product_id', 'type', 'air_temperature_k',
       'process_temperature_k', 'rotational_speed_rpm', 'torque_nm',
       'tool_wear_min', 'failure_type'],
      dtype='object')

In [222]:

print('Tipo que mais se repete',df['type'].mode())
print('Media do numero de rotaçoes',df['rotational_speed_rpm'].mean())
print('Falha que mais se repete',df['failure_type'].max())
#descrição geral
df.describe()

Tipo que mais se repete 0    L
dtype: object
Media do numero de rotaçoes 1537.4195290235489
Falha que mais se repete Tool Wear Failure


,udi,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min
count,6667.00,6667.00,6667.00,6667.00,6667.00,6667.00
mean,4994.59,299.99,309.99,1537.42,40.06,108.10
std,2896.13,1.99,1.49,177.18,9.95,63.36
min,1.00,295.30,305.70,1168.00,3.80,0.00
25%,2496.50,298.30,308.80,1422.50,33.20,54.00
50%,4983.00,300.00,310.00,1503.00,40.20,108.00
75%,7510.50,301.50,311.10,1612.00,46.80,162.00
max,10000.00,304.50,313.80,2886.00,76.60,251.00


In [223]:
# Importing important libraries

import pyforest
from lazypredict.Supervised import LazyRegressor
from pandas.plotting import scatter_matrix

# Scikit-learn packages
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error

# Setting up max columns displayes to 100
pd.options.display.max_columns = None
pd.set_option('display.width', None)

# Hide warnings
import warnings
warnings.filterwarnings("ignore")

# Set theme
sns.set_style('darkgrid')

<IPython.core.display.Javascript object>

In [224]:
#import MultiColumnLabelEncoder as le
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['type']=le.fit_transform(df['type'])
df['product_id']=le.fit_transform(df['product_id'])
df['failure_type']=le.fit_transform(df['failure_type'])

In [225]:
print(df)

        udi  product_id  type  air_temperature_k  process_temperature_k  \
0         1        4680     2             298.10                 308.60   
1         2         658     1             298.20                 308.70   
2         5         659     1             298.20                 308.70   
3         6        4681     2             298.10                 308.60   
4         7         660     1             298.10                 308.60   
...     ...         ...   ...                ...                    ...   
6662   9995        4679     1             298.80                 308.30   
6663   9996        6665     2             298.80                 308.40   
6664   9997         656     0             298.90                 308.40   
6665   9999         657     0             299.00                 308.70   
6666  10000        6666     2             299.00                 308.70   

      rotational_speed_rpm  torque_nm  tool_wear_min  failure_type  
0                     1551    

In [226]:
x=df.drop(columns='failure_type')
y = df.failure_type
x.head
y.head

<bound method NDFrame.head of 0       1
1       1
2       1
3       1
4       1
       ..
6662    1
6663    1
6664    1
6665    1
6666    1
Name: failure_type, Length: 6667, dtype: int64>

In [227]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=3,test_size=0.25)

print("Training set - Features: ", X_train.shape, "Target: ", y_train.shape)
print("Test set - Features: ", X_test.shape, "Target: ",y_test.shape)

Training set - Features:  (5000, 8) Target:  (5000,)
Test set - Features:  (1667, 8) Target:  (1667,)


In [228]:
from lazypredict.Supervised import LazyClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [229]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
print('Resultado')
result=print(models)

100%|██████████| 29/29 [00:07<00:00,  4.07it/s]

Resultado
                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
NearestCentroid                    0.43               0.59    None      0.58   
BaggingClassifier                  0.98               0.54    None      0.98   
DecisionTreeClassifier             0.98               0.52    None      0.98   
GaussianNB                         0.96               0.52    None      0.96   
XGBClassifier                      0.99               0.51    None      0.98   
QuadraticDiscriminantAnalysis      0.97               0.51    None      0.97   
LGBMClassifier                     0.98               0.46    None      0.98   
RandomForestClassifier             0.98               0.43    None      0.98   
SGDClassifier                      0.98               0.41    None      0.97   
ExtraTreeClassifier                0.96               0.40    None      0.96   
LogisticRegression            

In [230]:
result=pd.DataFrame(models)
result.columns
print(result)

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
NearestCentroid                    0.43               0.59    None      0.58   
BaggingClassifier                  0.98               0.54    None      0.98   
DecisionTreeClassifier             0.98               0.52    None      0.98   
GaussianNB                         0.96               0.52    None      0.96   
XGBClassifier                      0.99               0.51    None      0.98   
QuadraticDiscriminantAnalysis      0.97               0.51    None      0.97   
LGBMClassifier                     0.98               0.46    None      0.98   
RandomForestClassifier             0.98               0.43    None      0.98   
SGDClassifier                      0.98               0.41    None      0.97   
ExtraTreeClassifier                0.96               0.40    None      0.96   
LogisticRegression                 0.98 

In [231]:
print('Maior acuracia') 
result[result==result['Accuracy'].max()]

Maior acuracia


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,NaN,NaN,NaN,NaN,NaN
BaggingClassifier,NaN,NaN,NaN,NaN,NaN
DecisionTreeClassifier,NaN,NaN,NaN,NaN,NaN
GaussianNB,NaN,NaN,NaN,NaN,NaN
XGBClassifier,0.99,NaN,NaN,NaN,NaN
QuadraticDiscriminantAnalysis,NaN,NaN,NaN,NaN,NaN
LGBMClassifier,NaN,NaN,NaN,NaN,NaN
RandomForestClassifier,NaN,NaN,NaN,NaN,NaN
SGDClassifier,NaN,NaN,NaN,NaN,NaN


In [232]:

print('Menor tempo')
result[result==result['Time Taken'].min()]

Menor tempo


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,NaN,NaN,NaN,NaN,NaN
BaggingClassifier,NaN,NaN,NaN,NaN,NaN
DecisionTreeClassifier,NaN,NaN,NaN,NaN,NaN
GaussianNB,NaN,NaN,NaN,NaN,NaN
XGBClassifier,NaN,NaN,NaN,NaN,NaN
QuadraticDiscriminantAnalysis,NaN,NaN,NaN,NaN,NaN
LGBMClassifier,NaN,NaN,NaN,NaN,NaN
RandomForestClassifier,NaN,NaN,NaN,NaN,NaN
SGDClassifier,NaN,NaN,NaN,NaN,NaN


In [233]:
print('Maior F1 Score')
result[result==result['F1 Score'].max()]

Maior F1 Score


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,NaN,NaN,NaN,NaN,NaN
BaggingClassifier,NaN,NaN,NaN,NaN,NaN
DecisionTreeClassifier,NaN,NaN,NaN,NaN,NaN
GaussianNB,NaN,NaN,NaN,NaN,NaN
XGBClassifier,NaN,NaN,NaN,0.98,NaN
QuadraticDiscriminantAnalysis,NaN,NaN,NaN,NaN,NaN
LGBMClassifier,NaN,NaN,NaN,NaN,NaN
RandomForestClassifier,NaN,NaN,NaN,NaN,NaN
SGDClassifier,NaN,NaN,NaN,NaN,NaN


In [234]:
print('Maior Acuracia Balanceada')
result[result==result['Balanced Accuracy'].max()]

Maior Acuracia Balanceada


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,NaN,0.59,NaN,NaN,NaN
BaggingClassifier,NaN,NaN,NaN,NaN,NaN
DecisionTreeClassifier,NaN,NaN,NaN,NaN,NaN
GaussianNB,NaN,NaN,NaN,NaN,NaN
XGBClassifier,NaN,NaN,NaN,NaN,NaN
QuadraticDiscriminantAnalysis,NaN,NaN,NaN,NaN,NaN
LGBMClassifier,NaN,NaN,NaN,NaN,NaN
RandomForestClassifier,NaN,NaN,NaN,NaN,NaN
SGDClassifier,NaN,NaN,NaN,NaN,NaN


Com isso escolhi o XBG 

In [235]:
import xgboost as xgb
from xgboost import XGBClassifier

In [236]:
xgbr = xgb.XGBClassifier(verbosity=0) 


In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6667 entries, 0 to 6666
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   udi                    6667 non-null   int64  
 1   product_id             6667 non-null   int64  
 2   type                   6667 non-null   int64  
 3   air_temperature_k      6667 non-null   float64
 4   process_temperature_k  6667 non-null   float64
 5   rotational_speed_rpm   6667 non-null   int64  
 6   torque_nm              6667 non-null   float64
 7   tool_wear_min          6667 non-null   int64  
 8   failure_type           6667 non-null   int64  
dtypes: float64(3), int64(6)
memory usage: 468.9 KB


In [238]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=3,test_size=0.01)

print("Training set - Features: ", X_train.shape, "Target: ", y_train.shape)
print("Test set - Features: ", X_test.shape, "Target: ",y_test.shape)
xgbr.fit(X_train, y_train)

Training set - Features:  (6600, 8) Target:  (6600,)
Test set - Features:  (67, 8) Target:  (67,)


XGBClassifier(objective='multi:softprob', verbosity=0)

In [239]:
score = xgbr.score(X_train, y_train)  
print("Training score: ", score)

Training score:  0.993030303030303


In [240]:
predicted_y = xgbr.predict(X_test)
print(predicted_y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]


In [241]:
confer=pd.DataFrame(predicted_y,y_test)
print(confer)
corr_confer = confer.corr()
print(corr_confer)


              0
failure_type   
1             1
1             1
1             1
1             1
1             1
...          ..
1             1
1             1
1             1
1             1
1             1

[67 rows x 1 columns]
     0
0 1.00


In [242]:
df2=pd.read_csv ('desafio_manutencao_preditiva_teste.csv')
print(df2)
df2.columns

       udi product_id type  air_temperature_k  process_temperature_k  \
0      446     L47625    L             297.50                 308.60   
1     7076     L54255    L             300.70                 310.50   
2     1191     L48370    L             297.20                 308.40   
3     2618     L49797    L             299.40                 309.10   
4     5067     L52246    L             304.10                 313.10   
...    ...        ...  ...                ...                    ...   
3328  5554     L52733    L             302.50                 311.90   
3329  6961     L54140    L             300.70                 311.00   
3330  6914     L54093    L             300.80                 311.20   
3331  5510     L52689    L             302.80                 312.20   
3332  3066     M17925    M             300.10                 309.20   

      rotational_speed_rpm  torque_nm  tool_wear_min  
0                     1793      26.70             70  
1                     153

Index(['udi', 'product_id', 'type', 'air_temperature_k',
       'process_temperature_k', 'rotational_speed_rpm', 'torque_nm',
       'tool_wear_min'],
      dtype='object')

In [243]:
df2['type']=le.fit_transform(df2['type'])
df2['product_id']=le.fit_transform(df2['product_id'])
print(df2)

       udi  product_id  type  air_temperature_k  process_temperature_k  \
0      446         427     1             297.50                 308.60   
1     7076        1742     1             300.70                 310.50   
2     1191         577     1             297.20                 308.40   
3     2618         859     1             299.40                 309.10   
4     5067        1362     1             304.10                 313.10   
...    ...         ...   ...                ...                    ...   
3328  5554        1451     1             302.50                 311.90   
3329  6961        1720     1             300.70                 311.00   
3330  6914        1708     1             300.80                 311.20   
3331  5510        1443     1             302.80                 312.20   
3332  3066        2623     2             300.10                 309.20   

      rotational_speed_rpm  torque_nm  tool_wear_min  
0                     1793      26.70             70  
1

In [244]:
predicted_df2 = xgbr.predict(df2)
print(predicted_df2)
#df2['failure_type']=predicted_df2
print(df2)

[1 1 1 ... 1 1 1]
       udi  product_id  type  air_temperature_k  process_temperature_k  \
0      446         427     1             297.50                 308.60   
1     7076        1742     1             300.70                 310.50   
2     1191         577     1             297.20                 308.40   
3     2618         859     1             299.40                 309.10   
4     5067        1362     1             304.10                 313.10   
...    ...         ...   ...                ...                    ...   
3328  5554        1451     1             302.50                 311.90   
3329  6961        1720     1             300.70                 311.00   
3330  6914        1708     1             300.80                 311.20   
3331  5510        1443     1             302.80                 312.20   
3332  3066        2623     2             300.10                 309.20   

      rotational_speed_rpm  torque_nm  tool_wear_min  
0                     1793      26.70 

In [245]:
df2['type'] = le.inverse_transform(df2['type'])
df2['product_id']=le.inverse_transform(df2['product_id'])
dfo['failure_type']=le.fit_transform(dfo['failure_type'])
predicted_df2 = le.inverse_transform(predicted_df2)

In [246]:
df2['failure_type']=predicted_df2
print(df2)


       udi product_id    type  air_temperature_k  process_temperature_k  \
0      446     L47625  H29552             297.50                 308.60   
1     7076     L54255  H29552             300.70                 310.50   
2     1191     L48370  H29552             297.20                 308.40   
3     2618     L49797  H29552             299.40                 309.10   
4     5067     L52246  H29552             304.10                 313.10   
...    ...        ...     ...                ...                    ...   
3328  5554     L52733  H29552             302.50                 311.90   
3329  6961     L54140  H29552             300.70                 311.00   
3330  6914     L54093  H29552             300.80                 311.20   
3331  5510     L52689  H29552             302.80                 312.20   
3332  3066     M17925  H29560             300.10                 309.20   

      rotational_speed_rpm  torque_nm  tool_wear_min failure_type  
0                     1793     

In [247]:
df2.to_excel('result.xlsx')